## Support Vector Machines

In [1]:
from init import *

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import sklearn.svm as svm
from sklearn import metrics
from sklearn import datasets
from sklearn.decomposition import PCA
import pandas as pd

In [3]:
%matplotlib inline

In [4]:
X = train_features.reshape(50000, 3*32*32)
Xt = test_features.reshape(10000, 3*32*32)
y = train_labels.flatten()s
yt = test_labels.flatten()

In [12]:
def SVM_SVC(_X=None, _Xt=None):
    if _X is None:
        _X = X
    
    if _Xt is None:
        _Xt = Xt
    
    msg("[SVM SVC] Training")
    svc = svm.SVC(verbose=True)
    svc.fit(X, y)
    done()
    
    msg("[SVM SVC] Training Accuracy")
    X_pred = svc.predict(X)
    msg(metrics.accuracy_score(y, X_pred))
    done()
    
    msg("[SVM SVC] Testing Accuracy")
    Xt_pred = svc.predict(Xt)
    msg(metrics.accuracy_score(yt, Xt_pred))
    done()

In [ ]:
SVM_SVC()

[SVM SVC] Training ... [LibSVM]

In [ ]:
X

In [ ]:
svc = svm.SVC(verbose=True)
svc.fit(X, y)

In [5]:
res = pd.DataFrame([
        ['1', 10]
    ], columns=['Iterations', 'Accuracy'])
res

,Iterations,Accuracy
0,1,10
